In [1]:
import pandas as pd
import requests
from lxml import html
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/tourist_arrivals.csv',low_memory=False)
df.head()



,Year,Country,Arrivals,Pleasure/Vacation,Visiting_relations and friends,Business,MICE,Sports,Health/Aurweda,Education,...,officials,Others,male,female,Age(3-19),Age(20-29),Age(30-39),Age(40-49),Age(50-59),Age(60 &above)
0,2020,AMERICAS,127423,84331,37766,2768,1014,948,206,345,...,15,6,64203,63220,15543,13911,21623,18637,20249,37455
1,2020,North America,119681,77315,37432,2563,927,904,183,312,...,15,6,60537,59144,15156,12410,19051,17543,19401,36100
2,2020,Canada,48729,22280,25581,410,156,151,49,84,...,6,0,24573,24156,7192,4667,6677,7015,8959,14219
3,2020,Mexico,2120,1939,54,61,33,9,3,19,...,0,2,1038,1082,155,369,538,344,312,402
4,2020,United States,68832,53096,11 797,2092,738,744,131,209,...,9,4,34924,33908,7809,7374,11836,10204,10130,21479


In [ ]:
print('Number of missing rows in tourist_arrivals:',df.isna().sum().sum())


Number of missing rows in tourist_arrivals: 0


In [ ]:
print('Number of unique values in torist_arrivals:')
for col in df.columns:
    print(f"{col} - {df[col].nunique()}")

Number of unique values in torist_arrivals:
Year - 2
Country - 94
Arrivals - 205
Pleasure/Vacation - 207
Visiting_relations and friends - 163
Business - 144
MICE - 124
Sports - 96
Health/Aurweda - 81
Education - 69
Religious - 29
officials - 24
Others - 18
male - 208
female - 207
Age(3-19) - 185
Age(20-29) - 200
Age(30-39) - 203
Age(40-49) - 198
Age(50-59) - 189
Age(60 &above) - 190


In [ ]:
print('Columns change to integer:')
for col in df.columns:
  if(col != "Country" and col != "Year"):
    print(col)
    df[col] = df[col].astype(str).str.replace('[\ \,]',"").replace('O',"0").apply(pd.to_numeric,errors='raise').astype(int)


Columns change to integer:
Arrivals
Pleasure/Vacation
Visiting_relations and friends
Business
MICE
Sports
Health/Aurweda
Education
Religious
officials
Others
male
female
Age(3-19)
Age(20-29)
Age(30-39)
Age(40-49)
Age(50-59)
Age(60 &above)


<ipython-input-152-77e719ee4fdc>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].astype(str).str.replace('[\ \,]',"").replace('O',"0").apply(pd.to_numeric,errors='raise').astype(int)


In [ ]:
 print("Check any Naan values exits after the conversion")
 df.isna().any()


Check any Naan values exits after the conversion


Year                              False
Country                           False
Arrivals                          False
Pleasure/Vacation                 False
Visiting_relations and friends    False
                                  ...  
Age(20-29)                        False
Age(30-39)                        False
Age(40-49)                        False
Age(50-59)                        False
Age(60 &above)                    False
Length: 21, dtype: bool

In [ ]:

pd.set_option('display.max_rows', 20)
df

,Year,Country,Arrivals,Pleasure/Vacation,Visiting_relations and friends,Business,MICE,Sports,Health/Aurweda,Education,...,officials,Others,male,female,Age(3-19),Age(20-29),Age(30-39),Age(40-49),Age(50-59),Age(60 &above)
0,2020,AMERICAS,127423,84331,37766,2768,1014,948,206,345,...,15,6,64203,63220,15543,13911,21623,18637,20249,37455
1,2020,North America,119681,77315,37432,2563,927,904,183,312,...,15,6,60537,59144,15156,12410,19051,17543,19401,36100
2,2020,Canada,48729,22280,25581,410,156,151,49,84,...,6,0,24573,24156,7192,4667,6677,7015,8959,14219
3,2020,Mexico,2120,1939,54,61,33,9,3,19,...,0,2,1038,1082,155,369,538,344,312,402
4,2020,United States,68832,53096,11797,2092,738,744,131,209,...,9,4,34924,33908,7809,7374,11836,10204,10130,21479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2019,Oman,1045,983,54,8,0,0,0,0,...,0,0,589,456,327,271,248,123,53,23
206,2019,Qatar,129,115,7,1,5,0,1,0,...,0,0,94,35,21,17,37,26,16,12
207,2019,Saudi Arabia,4755,4432,286,1,0,3,33,0,...,0,0,2547,2208,1876,979,951,500,310,139
208,2019,Emirates,352,333,14,1,2,1,1,0,...,0,0,212,140,46,68,108,61,50,19


In [ ]:
df2 = df.copy()
df2 = df2.drop(['Country', 'Year'], axis=1)
(df2 < 0).any()


Arrivals                          False
Pleasure/Vacation                 False
Visiting_relations and friends    False
Business                          False
MICE                              False
Sports                            False
Health/Aurweda                    False
Education                         False
Religious                         False
officials                         False
Others                            False
male                              False
female                             True
Age(3-19)                         False
Age(20-29)                        False
Age(30-39)                         True
Age(40-49)                         True
Age(50-59)                         True
Age(60 &above)                     True
dtype: bool

In [ ]:
for cols in df2.columns:
  #print(df2[cols])
  if(df2[cols].lt(1).any()):
      print(cols)

Visiting_relations and friends
Business
MICE
Sports
Health/Aurweda
Education
Religious
officials
Others
female
Age(3-19)
Age(30-39)
Age(40-49)
Age(50-59)
Age(60 &above)


In [ ]:
#Replace negatives with 0
df2[df2 < 0] = 0